# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import os
import math
import time
import multiprocessing
from random import shuffle
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Principal Component Analysis of Left,Right,Head

In [11]:
X=[[521, 106], [511, 288], [364, 296], [251, 490], [487, 346], [668, 288], [660, 510], [592, 390], [430, 597], [585, 601], [485, 84], [549, 82], [434, 113], [589, 111]]
for x in range(len(X)):
    if(X[x]==None):
        X[x]=[1000,1000]
pca = sklearnPCA(n_components=1)
X=pca.fit_transform(X)
#pca.explained_variance_ratio_ #2% information loss
X

array([[-201.64004003],
       [ -19.37493551],
       [  -1.83578616],
       [ 199.09968628],
       [  40.06255713],
       [ -29.5810794 ],
       [ 192.46940212],
       [  77.14372284],
       [ 294.23705456],
       [ 288.15246439],
       [-221.2532431 ],
       [-227.40947898],
       [-188.99921262],
       [-201.07111152]])

In [12]:
left=[[156, 208], None, None, None, None, None, [308, 191], [300, 265], [293, 269], None, [284, 243], [272, 262], [268, 267], [260, 218], [259, 238], [251, 259], [245, 268], [250, 219], [243, 235], [236, 245], [227, 257]]
for x in range(len(left)):
    if(left[x]==None):
        left[x]=[0,0]
pca = sklearnPCA(n_components=1)
left=pca.fit_transform(left)
#pca.explained_variance_ratio_ 15% information loss

In [13]:
right=[[15, 180], [33, 178], [65, 172], [99, 169], [131, 161], [75, 123], [122, 130], None, None, [90, 109], [153, 129], None, None, [106, 106], [152, 130], None, None, [113, 109], [144, 130], [140, 138], None]
for x in range(len(right)):
    if(right[x]==None):
        right[x]=[0,0]
pca = sklearnPCA(n_components=1)
right=pca.fit_transform(right)
#pca.explained_variance_ratio_ #15% information loss

In [14]:
dfhead=pd.DataFrame(data=X)
dfhead=dfhead.T
dfhead=dfhead.rename(columns={0:"head_1",1:"head_2",2:"head_3",3:"head_4",4:"head_5",5:"head_6",6:"head_7",7:"head_8",8:"head_9",9:"head_10",10:"head_11",11:"head_12",12:"head_13",13:"head_14"})
dfhead

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,head_11,head_12,head_13,head_14
0,-201.64004,-19.374936,-1.835786,199.099686,40.062557,-29.581079,192.469402,77.143723,294.237055,288.152464,-221.253243,-227.409479,-188.999213,-201.071112


# Convert all values into a Pandas DataFrame

#### Transform Left Hand keypoints to a Pandas DataFrame

In [16]:
dfleft=pd.DataFrame(data=left)
dfleft=dfleft.T
dfleft=dfleft.rename(columns={0:"left_1",1:"left_2",2:"left_3",3:"left_4",4:"left_5",5:"left_6",6:"left_7",7:"left_8",8:"left_9",9:"left_10",10:"left_11",11:"left_12",12:"left_13",13:"left_14",14:"left_15",15:"left_16",16:"left_17",17:"left_18",18:"left_19",19:"left_20",20:"left_21"})
dfleft

,left_1,left_2,left_3,left_4,left_5,left_6,left_7,left_8,left_9,left_10,...,left_12,left_13,left_14,left_15,left_16,left_17,left_18,left_19,left_20,left_21
0,-3.358081,252.489634,252.489634,252.489634,252.489634,252.489634,-103.179776,-147.668362,-145.260297,252.489634,...,-125.1081,-125.57892,-86.375717,-99.251317,-107.677588,-109.404449,-79.727924,-85.484911,-87.159372,-88.729032


### Transform Right Hand Keypoints to a Pandas DataFrame

In [17]:
dfright=pd.DataFrame(data=right)
dfright=dfright.T
dfright=dfright.rename(columns={0:"right_1",1:"right_2",2:"right_3",3:"right_4",4:"right_5",5:"right_6",6:"right_7",7:"right_8",8:"right_9",9:"right_10",10:"right_11",11:"right_12",12:"right_13",13:"right_14",14:"right_15",15:"right_16",16:"right_17",17:"right_18",18:"right_19",19:"right_20",20:"right_21"})
dfright

,right_1,right_2,right_3,right_4,right_5,right_6,right_7,right_8,right_9,right_10,...,right_12,right_13,right_14,right_15,right_16,right_17,right_18,right_19,right_20,right_21
0,-34.467756,-44.094899,-59.295499,-78.089206,-91.723543,-27.140608,-61.849758,115.823782,115.823782,-25.504603,...,115.823782,115.823782,-33.104903,-80.505436,115.823782,115.823782,-39.80729,-75.530589,-79.308218,115.823782


### Concatenate the head,left,right dataframe and save it to a csv.

In [18]:
df=pd.concat([dfhead, dfleft,dfright], axis=1)
df=df[0:0]
df.to_csv("sample.csv",index=False)

In [20]:
import os

def buildGestureDict(path):
	if os.path.isdir(path):
		gestureDict = {}

		for item in os.listdir(path):
			subpath = os.path.join(path, item)
			if (os.path.isdir(subpath)):
				gesture = gestureDict[item] = {
					'videos': []
				}

				for item in os.listdir(subpath):
					if (item.lower().endswith('.mp4')):
						split = item.split('-')
						video = {
							'filepath': subpath+"/"+item, # with respect to the path argument
							'filename': item,
							'no': int(split[0]),
							'gesture': split[1],
							'actor': split[2].split('.')[0]
						}

						gesture['videos'].append(video)

		return gestureDict
    
sample=buildGestureDict("dataset/")
sample

{'1': {'videos': [{'filepath': 'dataset/1/1-1-jer.mp4',
    'filename': '1-1-jer.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'jer'},
   {'filepath': 'dataset/1/1-1-jolo.mp4',
    'filename': '1-1-jolo.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'jolo'},
   {'filepath': 'dataset/1/1-1-kyle.mp4',
    'filename': '1-1-kyle.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'kyle'},
   {'filepath': 'dataset/1/1-1-min.mp4',
    'filename': '1-1-min.mp4',
    'no': 1,
    'gesture': '1',
    'actor': 'min'}]},
 '2': {'videos': [{'filepath': 'dataset/2/1-2-jer.mp4',
    'filename': '1-2-jer.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'jer'},
   {'filepath': 'dataset/2/1-2-jolo.mp4',
    'filename': '1-2-jolo.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'jolo'},
   {'filepath': 'dataset/2/1-2-kyle.mp4',
    'filename': '1-2-kyle.mp4',
    'no': 1,
    'gesture': '2',
    'actor': 'kyle'},
   {'filepath': 'dataset/2/1-2-min.mp4',
    'filename': '1-2-min.mp4',
  

In [30]:
for x in sample:
    print(x+".csv")
    for video in  sample[x]["videos"]:
        filepath=video["filepath"]
        gesture=video["gesture"]
        speaker=video["actor"]

again.csv
all.csv
bad.csv
bathroom.csv
before.csv
big.csv
boy.csv
doctor.csv
father.csv
fine.csv
finish.csv
first.csv
friday.csv
friend.csv
girl.csv
goodbye.csv
good_1.csv
good_2.csv
hello.csv
help.csv
hospital.csv
hungry.csv
language.csv
later.csv
live.csv
love.csv
meet_1.csv
meet_2.csv
monday.csv
morning.csv
mother.csv
name.csv
night.csv
no.csv
nothing.csv
okay.csv
please.csv
ready_1.csv
ready_2.csv
saturday.csv
small.csv
sorry.csv
sunday.csv
teach.csv
thank you.csv
thankyou.csv
thursday.csv
today.csv
tomorrow.csv
tuesday.csv
understand.csv
wednesday.csv
welcome.csv
where.csv
yes.csv
you.csv


In [31]:
testing=pd.read_csv("testsave.csv")
testing

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
0,-45.598327,-150.024087,-12.268116,-123.150291,592.648717,-286.669179,-480.161243,-545.157230,592.648717,592.648717,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,Monday,Kalbo
1,153.183665,-9.134377,72.792640,-77.131622,-250.813036,-93.632597,-350.285875,-458.781987,-283.665699,625.047642,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,Monday,Kalbo
2,-257.252334,-62.034792,-59.908610,164.991826,228.300128,-60.082127,164.743944,357.580747,300.818789,306.944927,...,-28.219854,-34.746535,-37.371310,-19.540587,-19.138907,-23.176291,-27.011647,2.0,Monday,Kalbo
3,-5.939419,-101.366557,28.979405,-80.408256,-2.586001,-235.785831,-408.979006,-524.054760,635.476964,635.476964,...,-37.923946,-44.914255,-51.813803,2.043373,-14.900132,-21.544593,-26.038005,3.0,Monday,Kalbo
4,112.110388,-22.031792,80.281248,-120.040494,-35.864636,-123.031054,-337.083639,-487.810207,-258.550737,653.333624,...,-143.039737,-164.878824,-183.784952,-101.583447,-115.080412,-131.672631,-148.637956,4.0,Monday,Kalbo
5,108.081296,-20.421535,80.014678,-107.188126,-46.280841,-125.197584,-343.226732,-481.813832,-250.757951,655.659977,...,6.305895,-24.804994,-44.315058,44.880166,6.849808,-14.211785,-33.008497,5.0,Monday,Kalbo
6,-3.276345,-95.210267,39.666116,-99.679610,-36.700066,-231.332563,-407.601902,-520.696332,641.482070,641.482070,...,-184.234591,-209.353297,-225.163499,-139.994216,-151.014993,-177.419943,-188.838875,6.0,Monday,Kalbo
7,114.454000,-17.861565,80.268726,-129.959591,-69.279502,-120.232053,-327.345778,-483.367255,-253.309087,654.080462,...,-192.030968,-216.702241,-236.887828,-153.989834,-162.217309,-180.907667,-200.508886,7.0,Monday,Kalbo
8,112.976129,-18.245013,79.061014,-129.868975,-53.940850,-120.187341,-336.663298,-484.366134,-252.279729,654.009422,...,-138.727956,-162.466100,-179.527890,-96.913220,-113.375622,-128.882564,-146.929486,8.0,Monday,Kalbo
9,29.456970,-80.555567,40.769113,-101.061281,644.523222,-207.410963,-392.267172,-518.445008,-256.518026,644.523222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,Monday,Kalbo


In [17]:
path = 'gestures720/'
df=pd.read_csv("sample.csv")
for filename in os.listdir(path):
    #print(filename)
    dffile=pd.read_csv("gestures720/"+filename)
    df=df.append(dffile,sort=False)
df["frame"]=df["frame"].astype(int)
df[df["gesture"]==1]
#df.to_csv("dataset.csv",index=False)

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
0,-216.629445,-78.863662,-62.331491,133.997137,380.481566,-88.277196,110.699692,356.390206,239.546773,230.313245,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,jer
1,-212.548188,-74.819194,-59.018039,131.221680,323.571951,-83.509698,115.932987,361.654539,245.222152,239.721829,...,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,1,1,jer
2,-193.884186,-61.024351,-69.336202,116.503566,90.557602,-46.176213,160.093458,413.747847,242.971970,263.918711,...,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,2,1,jer
3,-176.524482,-47.555117,-81.925391,81.546239,-86.800888,-15.679804,188.668001,447.142804,243.690064,285.215812,...,-510.174501,-510.174501,-510.174501,639.211618,-510.174501,-510.174501,-510.174501,3,1,jer
4,-176.185181,-45.995061,-72.864577,82.776308,-117.510556,-16.278851,192.323229,446.809708,248.643469,285.270613,...,11.662280,-6.933117,-19.438155,-18.968448,-8.198719,-8.628352,-12.593842,4,1,jer
5,-179.157685,-49.359688,-78.679839,91.247465,-86.555326,-17.877787,188.248689,440.435146,247.042589,280.710297,...,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,953.585891,5,1,jer
6,-193.775431,-59.108088,-76.609224,115.446506,48.182479,-39.622641,165.183917,418.457369,245.763832,272.100998,...,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,6,1,jer
7,-208.440245,-67.842313,-61.093801,118.975006,253.156507,-72.591977,125.562657,376.433732,248.064787,246.464667,...,-17.893310,-33.972852,-50.993620,12.694120,-12.860322,-24.225704,-32.736088,7,1,jer
8,-221.946787,-78.620666,-58.415003,137.490577,405.538324,-95.629073,97.745027,347.489588,244.538778,228.181437,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8,1,jer
9,-220.777922,-78.450350,-67.741767,133.987124,376.005472,-87.998824,109.707088,362.214110,239.794635,237.911292,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9,1,jer


In [18]:
path = 'datasetpending/'
#df=pd.read_csv("sample.csv")
for filename in os.listdir(path):
    #print(filename)
    dffile=pd.read_csv("datasetpending/"+filename)
    df=df.append(dffile,sort=False)
df["frame"]=df["frame"].astype(int)
df[df["gesture"]==1].reset_index()
#df.to_csv("dataset.csv",index=False)

,index,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
0,0,-216.629445,-78.863662,-62.331491,133.997137,380.481566,-88.277196,110.699692,356.390206,239.546773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,jer
1,1,-212.548188,-74.819194,-59.018039,131.221680,323.571951,-83.509698,115.932987,361.654539,245.222152,...,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,1,1,jer
2,2,-193.884186,-61.024351,-69.336202,116.503566,90.557602,-46.176213,160.093458,413.747847,242.971970,...,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,2,1,jer
3,3,-176.524482,-47.555117,-81.925391,81.546239,-86.800888,-15.679804,188.668001,447.142804,243.690064,...,-510.174501,-510.174501,-510.174501,639.211618,-510.174501,-510.174501,-510.174501,3,1,jer
4,4,-176.185181,-45.995061,-72.864577,82.776308,-117.510556,-16.278851,192.323229,446.809708,248.643469,...,11.662280,-6.933117,-19.438155,-18.968448,-8.198719,-8.628352,-12.593842,4,1,jer
5,5,-179.157685,-49.359688,-78.679839,91.247465,-86.555326,-17.877787,188.248689,440.435146,247.042589,...,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,953.585891,5,1,jer
6,6,-193.775431,-59.108088,-76.609224,115.446506,48.182479,-39.622641,165.183917,418.457369,245.763832,...,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,6,1,jer
7,7,-208.440245,-67.842313,-61.093801,118.975006,253.156507,-72.591977,125.562657,376.433732,248.064787,...,-17.893310,-33.972852,-50.993620,12.694120,-12.860322,-24.225704,-32.736088,7,1,jer
8,8,-221.946787,-78.620666,-58.415003,137.490577,405.538324,-95.629073,97.745027,347.489588,244.538778,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8,1,jer
9,9,-220.777922,-78.450350,-67.741767,133.987124,376.005472,-87.998824,109.707088,362.214110,239.794635,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9,1,jer


In [19]:
df.to_csv("dataset1-6.csv",index=False)

In [20]:
df=pd.read_csv("dataset1-6.csv")

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
0,-216.629445,-78.863662,-62.331491,133.997137,380.481566,-88.277196,110.699692,356.390206,239.546773,230.313245,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1.0,jer
1,-212.548188,-74.819194,-59.018039,131.221680,323.571951,-83.509698,115.932987,361.654539,245.222152,239.721829,...,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,-275.200497,1,1.0,jer
2,-193.884186,-61.024351,-69.336202,116.503566,90.557602,-46.176213,160.093458,413.747847,242.971970,263.918711,...,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,-234.683771,2,1.0,jer
3,-176.524482,-47.555117,-81.925391,81.546239,-86.800888,-15.679804,188.668001,447.142804,243.690064,285.215812,...,-510.174501,-510.174501,-510.174501,639.211618,-510.174501,-510.174501,-510.174501,3,1.0,jer
4,-176.185181,-45.995061,-72.864577,82.776308,-117.510556,-16.278851,192.323229,446.809708,248.643469,285.270613,...,11.662280,-6.933117,-19.438155,-18.968448,-8.198719,-8.628352,-12.593842,4,1.0,jer
5,-179.157685,-49.359688,-78.679839,91.247465,-86.555326,-17.877787,188.248689,440.435146,247.042589,280.710297,...,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,-288.377574,953.585891,5,1.0,jer
6,-193.775431,-59.108088,-76.609224,115.446506,48.182479,-39.622641,165.183917,418.457369,245.763832,272.100998,...,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,-58.437109,6,1.0,jer
7,-208.440245,-67.842313,-61.093801,118.975006,253.156507,-72.591977,125.562657,376.433732,248.064787,246.464667,...,-17.893310,-33.972852,-50.993620,12.694120,-12.860322,-24.225704,-32.736088,7,1.0,jer
8,-221.946787,-78.620666,-58.415003,137.490577,405.538324,-95.629073,97.745027,347.489588,244.538778,228.181437,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8,1.0,jer
9,-220.777922,-78.450350,-67.741767,133.987124,376.005472,-87.998824,109.707088,362.214110,239.794635,237.911292,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9,1.0,jer


In [ ]:
data=pd.read_csv("dataset1-6.csv")
gestures=data["gesture"].unique()
#data=data[data["speaker"]!="ling_f"]
speakers=data["speaker"].unique().tolist()2
data[data["gesture"]==1]["speakers"]

In [36]:
def train_all(df):
    models={}
    words=df["gesture"].unique().tolist()
    for word in words:
        dataword=df[df["gesture"]==word]
        speakers=dataword["speaker"].unique().tolist()
        lengths=[]
        for speaker in speakers:
            lengths.append(len(dataword[dataword["speaker"]==speaker]))
        dataword=dataword.drop(columns=[dataword.columns[56],dataword.columns[57],dataword.columns[58]])
        #BAYESIAN INFORMATION CRITERION FOR SELECTING THE BEST MODEL
        best_score,best_model=float("inf"),None
        models[word]=GaussianHMM(n_components=2,covariance_type="diag", n_iter=1000).fit(dataword,lengths)
        for x in range(2,20):
            try:
                model=GaussianHMM(n_components=x,covariance_type="diag", n_iter=1000).fit(dataword,lengths)
                logL=models[word].score(dataword,lengths)
                n_features=dataword.shape[1]
                n_params =x * (x - 1) + 2 * n_features * x
                logN = np.log(self.X.shape[0])
                bic = -2 * logL + n_params * logN
                if bic < best_score:
                    best_score, best_model = bic, model
            except Exception as e:
                continue
        models[word]=best_model
    return models

In [37]:
def recognize(dictModels,data):
    probmodels={}
    prob=float("-inf")
    word=None
    data=data.drop(columns=[data.columns[56],data.columns[57],data.columns[58]])
    for modelword,model in dictModels.items():
        try:
            modelprob=model.score(data)
            probmodels[modelword]=modelprob
            if(modelprob>prob):
                prob=modelprob
                word=modelword
        except Exception as e:
            continue
    #print(probmodels)
    return word

## 60%-40% Testing

In [38]:
data=pd.read_csv("dataset1-6.csv")
training={}
words=data["gesture"].unique()
totalTrue=0
totalTest=0
start=time.time()
for word in words:
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    shuffle(speakers)
    datalength=len(speakers)
    if(datalength%2==0):
        traininglength=math.floor((datalength*.0))+1
        testlength=round((datalength*.40))
    else:
        traininglength=math.floor((datalength*.60))+1
        testlength=round((datalength*.40))-1
    trainingspeaker=speakers[0:traininglength]
    testspeakers=speakers[datalength-testlength:datalength]
    totalTest=totalTest+len(testspeakers)
    trainingdata=data[data["gesture"]!=word]
    for speaker in trainingspeaker:
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]==speaker])
    training=train_all(trainingdata)
    print("Finished Training the model for:"+word)
    #print(training)
    for testspeaker in testspeakers:
        print("Recognition for "+word+" speaker is: "+testspeaker)
        guessword=recognize(training,dataperword[dataperword["speaker"]==testspeaker])
        if(guessword==word):
            true=true+1
            totalTrue=totalTrue+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(testspeakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))
print("Total Accuracy"+str(totalTrue/totalTest))
print("Took "+str(time.time()-start)+" seconds")

Finished Training the model for:1.0
Recognition for 1.0 speaker is: min
Recognition for 1.0 speaker is: jer
Recognized 1.0:0 out of 2
Finished Training the model for:2.0
Recognition for 2.0 speaker is: jer
Recognition for 2.0 speaker is: jolo
Recognized 2.0:0 out of 2
Finished Training the model for:3.0
Recognition for 3.0 speaker is: jolo
Recognition for 3.0 speaker is: jer
Recognized 3.0:0 out of 2
Finished Training the model for:4.0
Recognition for 4.0 speaker is: kyle
Recognition for 4.0 speaker is: min
Recognized 4.0:0 out of 2
Finished Training the model for:5.0
Recognition for 5.0 speaker is: jolo
Recognition for 5.0 speaker is: jer
Recognized 5.0:0 out of 2
Finished Training the model for:6.0
Recognition for 6.0 speaker is: kyle
Recognition for 6.0 speaker is: jolo
Recognized 6.0:0 out of 2
Finished Training the model for:7.0
Recognition for 7.0 speaker is: min
Recognition for 7.0 speaker is: kyle
Recognized 7.0:0 out of 2
Finished Training the model for:8.0
Recognition for 8.0

KeyboardInterrupt: 

## Cross-Validation

In [8]:
training={}
words=data["gesture"].unique()
totalTrue=0
totalTest=0
start=time.time()
for word in words:
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    datalength=len(speakers)
    totalTest=totalTest+datalength
    for index in range(datalength):
        trainingdata=data[data["gesture"]!=word]
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]!=speakers[index]])
        training=train_all(trainingdata)
        guessword=recognize(training,dataperword[dataperword["speaker"]==speakers[index]])
        if(guessword==word):
            true=true+1
            totalTrue=totalTrue+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(speakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))
print("Total Accuracy"+str(totalTrue/totalTest))
print("Took "+str(time.time()-start)+" seconds")

Recognized again:0 out of 6
Recognized big:0 out of 5
Recognized boy:0 out of 8
Recognized doctor:0 out of 6
Recognized father:0 out of 5
Recognized fine:0 out of 7
Recognized friday:0 out of 5
Recognized friend:3 out of 8
Recognized good_2:0 out of 5
Recognized hello:0 out of 6
Recognized hospital:0 out of 5
Recognized hungry:0 out of 5
Recognized language:0 out of 5
Recognized live:1 out of 5
Recognized love:0 out of 6
Recognized morning:0 out of 5
Recognized mother:0 out of 5
Recognized name:0 out of 5
Recognized night:0 out of 6
Recognized no:0 out of 7
Recognized please:0 out of 7
Recognized saturday:0 out of 5
Recognized small:0 out of 6
Recognized sorry:3 out of 7
Recognized teach:0 out of 6
Recognized thank you:0 out of 6
Recognized tomorrow:0 out of 7
Recognized understand:2 out of 7
Recognized wednesday:0 out of 6
Recognized where:2 out of 8
Recognized yes:0 out of 9
Recognized you:0 out of 7
Total Accuracy0.05612244897959184
Took 965.3825385570526 seconds


## Cross Validation with MultiProcessing

In [4]:
def cross_validate(word):
    data=pd.read_csv("dataset.csv")
    true=0
    traininglength=0
    testlength=0
    dataperword=data[data["gesture"]==word]
    speakers=dataperword["speaker"].unique().tolist()
    datalength=len(speakers)
    totalTest=totalTest+datalength
    for index in range(datalength):
        trainingdata=data[data["gesture"]!=word]
        trainingdata=trainingdata.append(dataperword[dataperword["speaker"]!=speakers[index]])
        training=train_all(trainingdata)
        guessword=recognize(training,dataperword[dataperword["speaker"]==speakers[index]])
        if(guessword==word):
            true=true+1
    print("Recognized "+word+":"+ str(true)+" out of "+str(len(speakers)))
    #print("Total Data: "+str(datalength)+" Training Set: "+str(0)+"-"+str(traininglength)+" Test Set:"+str(datalength-testlength)+"-"+str(datalength-1))


In [ ]:
training={}
words=data["gesture"].unique().tolist()
totalTrue=0
totalTest=0
starttime = time.time()
pool = multiprocessing.Pool()
pool.map(cross_validate,words)
pool.close()
print('That took {} seconds'.format(time.time() - starttime))
#print("Total Accuracy"+str(totalTrue/totalTest))

In [ ]:
import time
import multiprocessing 

def basic_func(x):
    if x == 0:
        return 'zero'
    elif x%2 == 0:
        return 'even'
    else:
        return 'odd'

def multiprocessing_func(x):
    y = x*x
    #time.sleep(2)
    print('{} squared results in a/an {} number'.format(x, basic_func(y)))
    
    
def main():
    
    starttime = time.time()
    pool = multiprocessing.Pool()
    pool.map(multiprocessing_func, range(0,10))
    pool.close()
    print('That took {} seconds'.format(time.time() - starttime))
    
main()

In [22]:
data720=pd.read_csv("dataset720.csv")
data720[data720["gesture"]=="hello"]

,head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8,head_9,head_10,...,right_15,right_16,right_17,right_18,right_19,right_20,right_21,frame,gesture,speaker
1719,-202.551360,-55.162023,-69.888216,92.905244,153.155265,-38.867674,159.631017,372.778702,224.393472,235.256691,...,-1.674693,-15.695074,-28.246229,33.461634,17.937877,6.354823,-5.729357,0.0,hello,jer
1720,-162.199414,-31.844406,-90.821576,9.284754,-191.989705,28.215396,231.277575,437.981029,209.743286,269.559725,...,767.282159,776.215042,-384.348416,781.151416,-384.348416,-384.348416,-384.348416,1.0,hello,jer
1721,-151.053983,-24.148468,-86.925634,-62.379484,-224.119791,39.177396,241.783784,447.853698,210.481984,278.440334,...,-10.592419,-33.371834,-52.199731,23.972387,-2.249145,-19.548838,-34.618072,2.0,hello,jer
1722,-152.143958,-21.317795,-84.342431,-73.701074,-234.344105,37.880837,244.441780,447.288732,216.129996,279.806890,...,-12.269070,-34.491898,-51.496707,26.815834,-0.599895,-15.817787,-30.431668,3.0,hello,jer
1723,-146.091867,-18.405356,-87.772378,-78.942747,-271.772352,45.048204,247.229110,450.770983,214.116224,282.993084,...,-526.885677,720.526690,732.269728,-526.885677,-526.885677,-526.885677,-526.885677,4.0,hello,jer
1724,-149.786159,-22.306465,-90.491380,-58.247263,-245.837437,41.544526,243.614987,449.750777,206.496491,277.054152,...,-9.927241,-29.788335,-55.848250,25.390660,10.362365,-1.586900,-10.538659,5.0,hello,jer
1725,-167.421241,-34.426864,-89.621287,12.092602,-151.311723,22.611626,227.588758,429.841568,208.828107,267.494325,...,-453.292892,748.367993,739.806954,-453.292892,-453.292892,-453.292892,-453.292892,6.0,hello,jer
1726,-200.902373,-50.884028,-68.563382,101.783871,71.949368,-29.020779,173.223725,379.861223,229.300355,245.246623,...,-21.947045,-47.978712,-67.706956,3.206102,-22.979351,-42.460568,-52.601990,7.0,hello,jer
1727,-224.479163,-66.088226,-47.587991,134.186919,343.936030,-74.152239,119.986010,314.384701,230.766371,211.522709,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.0,hello,jer
1728,-227.325390,-69.706943,-56.403611,132.752278,358.298870,-72.968197,121.600303,319.946409,228.001470,213.252502,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,hello,jer


In [2]:
df=pd.read_csv("dataset1-6.csv")
df["speaker"].unique().tolist()

['jer',
 'jolo',
 'kyle',
 'min',
 'emman',
 'sab',
 'aslu_m_1',
 'sschool_f_2',
 'ling_f_1',
 'scott_m_1',
 'sschool_m_5',
 'sschool_f_1',
 'sschool_m_6',
 'sschool_m_3',
 'sschool_f_4',
 'sschool_f_3',
 'sschool_m_1',
 'sschool_f_5']

In [42]:
hello=df[df["gesture"]=="hello"]
yes=df[df["gesture"]=="yes"]

hellotrain=hello[hello["speaker"]!="min"]
hellotest=hello[hello["speaker"]=="min"]
speakers=hellotrain["speaker"].unique().tolist()
hellotest=hellotest.drop(columns=[hellotest.columns[56],hellotest.columns[57],hellotest.columns[58]])
lengths=[]
for speaker in speakers:
    lengths.append(len(hellotrain[hellotrain["speaker"]==speaker]))
    
hellotrain=hellotrain.drop(columns=[hellotrain.columns[56],hellotrain.columns[57],hellotrain.columns[58]])
best_modelHELLO=GaussianHMM(n_components=3, n_iter=1000).fit(hellotrain,lengths)
best_score=float("inf")
for x in range(2,30):
    try:
        helloHMM=GaussianHMM(n_components=x, n_iter=1000).fit(hellotrain,lengths)
        logL=helloHMM.score(hellotrain,lengths)
        n_features=hellotrain.shape[1]
        n_params =x * (x - 1) + 2 * n_features * x
        logN = np.log(hellotrain.shape[0])
        bic = -2 * logL + n_params * logN
        if bic < best_score:
            best_score, best_modelHELLO = bic, helloHMM
    except Exception as e:
        break
        
print("DONE FOR HELLO")
speakers=yes["speaker"].unique().tolist()
lengthsyes=[]
for speaker in speakers:
    lengthsyes.append(len(yes[yes["speaker"]==speaker]))
    
yes=yes.drop(columns=[yes.columns[56],yes.columns[57],yes.columns[58]])
best_modelYES=GaussianHMM(n_components=3,covariance_type="diag", n_iter=1000).fit(yes,lengthsyes)
best_score=float("inf")
for x in range(2,30):
    try:
        yesHMM=GaussianHMM(n_components=x,covariance_type="diag", n_iter=1000).fit(yes,lengthsyes)
        logL=yesHMM.score(yes,lengths)
        n_features=yesHMM.shape[1]
        n_params =x * (x - 1) + 2 * n_features * x
        logN = np.log(yesHMM.shape[0])
        bic = -2 * logL + n_params * logN
        if bic < best_score:
            best_score, best_modelYES = bic, yesHMM
    except Exception as e:
        break
print("DONE FOR YES")
best_modelHELLO=GaussianHMM(n_components=15, n_iter=1000).fit(hellotrain,lengths)
best_modelYES=GaussianHMM(n_components=15,covariance_type="diag", n_iter=1000).fit(yes,lengthsyes)
print(best_modelHELLO)
print(best_modelYES)

DONE FOR HELLO
DONE FOR YES
GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=15, n_iter=1000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)
GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=15, n_iter=1000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)


In [43]:
print(best_modelHELLO.score(hellotest,[len(hellotest)]))
print(best_modelYES.score(hellotest,[len(hellotest)]))

-3956.7445660376966
-5888.247179309779


In [44]:
dataDict={'1.0': -15261.38427154475, '2.0': -9437.87070293576, '3.0': -13960.787658345362, '4.0': -25913.403907365773, '5.0': -34009.30721816705, '6.0': -21744.525774253074, '7.0': -22300.092874599115, '8.0': -23404.15032439914, '9.0': -20863.443825746795, 'a': -15843.744109481857, 'after': -4924.416731688826, 'again': -4898.784170635769, 'all': -5034.136380679447, 'b': -5012.609320751689, 'bad': -16052.384046840363, 'bathroom': -17801.307129728357, 'before': -5122.9003940925595, 'big': -4904.469958110332, 'boy': -152244676285.64624, 'c': -6245.779484702058, 'd': -5033.1074788008655, 'e': -106933761594.81038, 'f': -106027543100.55899, 'father': -22331.9519969774, 'fine': -13862.333480122581, 'finish': -5287.398858917977, 'first': -43344.65130289363, 'friday': -8383.216245301746, 'friend': -5165.508505125129, 'g': -24928.022723183145, 'girl': -35533.60750785122, 'good': -4988.300737884949, 'goodbye': -20461.345819842303, 'h': -8486.08473667237, 'hello': -56467.54958816265, 'help': -4875.924023457175, 'hospital': -49162.027129326016, 'hungry': -5700.439322125796, 'i': -17349.661838039305, 'j': -14520.764502425183, 'k': -42771.34285619349, 'l': -31179.395782929907, 'language': -4895.8069199966685, 'last': -5084.640905872851, 'later': -17769.751020819454, 'like': -24490.47677539281, 'live': -4932.417810244127, 'love': -4887.381311319825, 'm': -20256.871057146527, 'meet': -4985.67046582599, 'monday': -6469.621718893153, 'morning': -4824.542345361398, 'mother': -4999.6011962282155, 'n': -17962.272243778516, 'name': -5083.0957319199915, 'night': -4904.1383785502885, 'no': -36238.90441290401, 'nothing': -4909.014229690761, 'o': -18347.123540696928, 'okay': -17505.144833213857, 'p': -17643.726236299288, 'please': -17928.49079569365, 'q': -18186.417618764983, 'r': -5432.408206790539, 'ready': -4872.188405630047, 's': -19597.717933963326, 'saturday': -7513.268586075117, 'small': -4851.440372783125, 'sorry': -5045.214457789473, 'start': -5047.692005156146, 'sunday': -4990.458848902605, 't': -16910.619663692003, 'teach': -4920.566948512222, 'thank you': -15945.410578100706, 'thursday': -15864.19299409022, 'today': -5133.761669637047, 'tomorrow': -5195.636546909891, 'tuesday': -17961.689165286272, 'u': -44783.89691385196, 'understand': -36575.165232038846, 'v': -18375.64517478299, 'w': -21695.774765833758, 'wednesday': -22963.592399396894, 'welcome': -4883.376762859122, 'where': -25790.288420803718, 'x': -16810.204158410754, 'y': -19176.79938501117, 'yes': -7034.405327070988, 'you': -20516.804934896933, 'z': -21635.38068425545, '1': -16024.966621277274, '2': -29583.180438032014, '3': -63414.687766861236, '4': -94173.32352331877, '5': -18906.44609009296, '6': -55424.38424973455, '7': -16468.191240106975, '8': -16173.113061207438, '9': -16217.763409383238, 'thankyou': -39873608085.77002, 'doctor': -4867.381632355824}
dataDict

{'1.0': -15261.38427154475,
 '2.0': -9437.87070293576,
 '3.0': -13960.787658345362,
 '4.0': -25913.403907365773,
 '5.0': -34009.30721816705,
 '6.0': -21744.525774253074,
 '7.0': -22300.092874599115,
 '8.0': -23404.15032439914,
 '9.0': -20863.443825746795,
 'a': -15843.744109481857,
 'after': -4924.416731688826,
 'again': -4898.784170635769,
 'all': -5034.136380679447,
 'b': -5012.609320751689,
 'bad': -16052.384046840363,
 'bathroom': -17801.307129728357,
 'before': -5122.9003940925595,
 'big': -4904.469958110332,
 'boy': -152244676285.64624,
 'c': -6245.779484702058,
 'd': -5033.1074788008655,
 'e': -106933761594.81038,
 'f': -106027543100.55899,
 'father': -22331.9519969774,
 'fine': -13862.333480122581,
 'finish': -5287.398858917977,
 'first': -43344.65130289363,
 'friday': -8383.216245301746,
 'friend': -5165.508505125129,
 'g': -24928.022723183145,
 'girl': -35533.60750785122,
 'good': -4988.300737884949,
 'goodbye': -20461.345819842303,
 'h': -8486.08473667237,
 'hello': -56467.5